In [378]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [379]:
positions = pd.read_csv("portfolio_positions_exploded.csv")
costs = pd.read_csv("portfolio_costs_exploded.csv")

Construção do PNL de Posições

DTD

In [380]:
positions['overview_date'] = pd.to_datetime(positions['overview_date'])
positions = positions.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])

positions['exposure_value_ontem'] = (
    positions.groupby(['portfolio_id','instrument_id','portfolio_origem'])['exposure_value']
    .shift(1)
)

positions['portfolio_nav_ontem'] = (
    positions.groupby(['portfolio_id'])['portfolio_nav']
    .shift(1)
)

positions = positions.dropna()

positions['dtd_ativo_pct'] = positions['dtd_ativo_fin'] / positions['exposure_value_ontem']
positions['dtd_carteira_pct'] = positions['dtd_ativo_fin'] / positions['portfolio_nav_ontem']

MTD

In [381]:
positions['Year'] = positions['overview_date'].dt.year
positions['Month'] = positions['overview_date'].dt.month

# Calcula ano/mês do mês anterior
positions['Month_after'] = positions['Month'] + 1
positions['Year_after'] = positions['Year']
positions.loc[positions['Month_after'] == 13, 'Month_after'] = 1
positions.loc[positions['Month'] == 12, 'Year_after'] += 1

# Tabela com último exposure_value do mês para cada grupo
last_prev_month = (
    positions
    .groupby(['portfolio_id','instrument_id','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','instrument_id','portfolio_origem','Year_after','Month_after','exposure_value']]

last_prev_month = last_prev_month.rename(
    columns={'Year_after':'Year','Month_after':'Month','exposure_value':'exposure_value_prev'}
)

# Faz merge para trazer o exposure_value_prev para cada linha
positions = positions.merge(
    last_prev_month,
    on=['portfolio_id','instrument_id','portfolio_origem','Year','Month'],
    how='left'
)

# Calcula o PnL MTD pra qualquer dia:
positions['mtd_ativo_fin'] = positions.groupby(
    ['portfolio_id', 'instrument_id', 'portfolio_origem', 'Year', 'Month']
)['dtd_ativo_fin'].cumsum()

positions['mtd_ativo_pct'] = positions['mtd_ativo_fin'] / positions['exposure_value_prev']

positions['mtd_carteira_pct'] = positions['mtd_ativo_fin'] / positions['portfolio_nav_ontem']

Expansão do PNL MTD para ativos que foram zerados ao longo do mês

In [382]:
# PnL MTD
group_cols = ['portfolio_id','instrument_id','portfolio_origem','Year','Month']
all_dates = positions.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (positions['overview_date'].dt.year == year) & (positions['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(positions['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Passo 2: Merge calendar com positions
positions_full = pd.merge(calendar, positions, on=group_cols + ['overview_date'], how='left')

# Passo 3: Forward fill do PnL e das outras colunas por ativo/mês
positions_full = positions_full.sort_values(group_cols + ['overview_date'])
positions_full['mtd_carteira_pct'] = positions_full.groupby(group_cols)['mtd_carteira_pct'].ffill()

In [383]:
# Tratamento do DF para exportação
positions_full_filtrado = positions_full[[
    'portfolio_id','portfolio_origem','overview_date','instrument_id',
    'dtd_ativo_fin','dtd_ativo_pct','dtd_carteira_pct', 'mtd_ativo_fin',
    'mtd_ativo_pct','mtd_carteira_pct']]
positions_full_filtrado.replace([np.inf, -np.inf], 0, inplace=True)

positions_full_filtrado.to_csv("positions_pnl_history.csv", index=False)


Construção do PNL de Custos

In [384]:
costs['overview_date'] = pd.to_datetime(costs['overview_date'])

costs['Year'] = costs['overview_date'].dt.year
costs['Month'] = costs['overview_date'].dt.month

costs = costs.sort_values(['origin_portfolio_id', 'overview_date'])

# Calcula ano/mês do mês anterior
costs['Month_after'] = costs['Month'] + 1
costs['Year_after'] = costs['Year']
costs.loc[costs['Month_after'] == 13, 'Month_after'] = 1
costs.loc[costs['Month'] == 12, 'Year_after'] += 1

# Calcula o PnL MTD pra qualquer dia:
costs['mtd_custos_fin'] = costs.groupby(
    ['category_name','origin_portfolio_id','root_portfolio','Year','Month']
)['dtd_custos_fin'].cumsum()

costs['dtd_custos_pct'] = costs['dtd_custos_fin'] / costs['portfolio_nav_ontem']
costs['mtd_custos_pct'] = costs['mtd_custos_fin'] / costs['portfolio_nav_ontem']

In [385]:
costs.replace([np.inf, -np.inf], 0, inplace=True)
costs.to_csv("portfolio_costs_exploded.csv", index=False)

In [386]:
df_filtrado = positions_full[
    (positions_full['portfolio_id'] == 1175)
]

df_filtrado.to_clipboard()

In [387]:
df_filtrado = costs[
    (costs['root_portfolio'] == 1247)
]

df_filtrado.to_clipboard()